# E. Coli exploration

**Author: T. Ruokokoski**

This notebook loads and inspects simple textbook model of E. Coli.

In [1]:
from escher import Builder
import cobra

model = cobra.io.load_model("textbook")

# 2) Create an Escher Builder with the built-in core map
b = Builder(
    model_json      = model,
    map_name        = "ecoli_core.Core metabolism",
    reaction_scale_width = 1.0   # optional: how thick arrows are
)

b

Exception: Could not find the map ecoli_core.Core metabolism on the server